In [ ]:
!pip install torch torchvision torchaudio matplotlib --quiet
!pip install pillow requests --quiet
!pip install numpy --quiet
!pip install pandas scikit-learn --quiet

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import os
import pandas as pd
import json
import requests
from PIL import Image
from io import BytesIO
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

Set device


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cpu


Dataset class




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os # Added missing import

ROOT_DIR = "/content/drive/MyDrive/Datasets/data/"

CSV_PATH = os.path.join(ROOT_DIR, "table.csv")
IMG_DIR  = ROOT_DIR # Changed from os.path.join(ROOT_DIR, "image")

print("CSV_PATH:", CSV_PATH)
print("IMG_DIR:", IMG_DIR)

Mounted at /content/drive
CSV_PATH: /content/drive/MyDrive/Datasets/data/table.csv
IMG_DIR: /content/drive/MyDrive/Datasets/data/


In [ ]:
df = pd.read_csv(CSV_PATH)
print("columens in table.csv:")
print(df.columns)
print("\nFirst 5 rows: ")
display(df.head())

columens in table.csv:
Index(['ndc11', 'rxcui', 'name', 'rxnavImageObjectId', 'rxnavImageFileName',
       'nlmImageFileName', 'RXNAV 120', 'RXBASE 120', 'RXNAV 300',
       'RXBASE 300', 'RXNAV 600', 'RXBASE 600', 'RXNAV 800', 'RXBASE 800',
       'RXNAV 1024', 'RXBASE 1024', 'RXNAV ORIGINAL', 'RXBASE ORIGINAL'],
      dtype='object')

First 5 rows: 


,ndc11,rxcui,name,rxnavImageObjectId,rxnavImageFileName,nlmImageFileName,RXNAV 120,RXBASE 120,RXNAV 300,RXBASE 300,RXNAV 600,RXBASE 600,RXNAV 800,RXBASE 800,RXNAV 1024,RXBASE 1024,RXNAV ORIGINAL,RXBASE ORIGINAL
0,00093-0311-01,978006,Loperamide Hydrochloride 2 MG Oral Capsule,185643662,00093-0311-01_RXNAVIMAGE10_26211358.jpg,00093-0311-01_NLMIMAGE10_6315B1FD.jpg,image/images/gallery/120/00093-0311-01_RXNAVIM...,image/images/gallery/120/00093-0311-01_NLMIMAG...,image/images/gallery/300/00093-0311-01_RXNAVIM...,image/images/gallery/300/00093-0311-01_NLMIMAG...,image/images/gallery/600/00093-0311-01_RXNAVIM...,image/images/gallery/600/00093-0311-01_NLMIMAG...,image/images/gallery/800/00093-0311-01_RXNAVIM...,image/images/gallery/800/00093-0311-01_NLMIMAG...,image/images/gallery/1024/00093-0311-01_RXNAVI...,image/images/gallery/1024/00093-0311-01_NLMIMA...,image/images/gallery/original/00093-0311-01_RX...,image/images/gallery/original/00093-0311-01_NL...
1,00093-3165-01,197985,Minocycline 50 MG Oral Capsule,185646490,00093-3165-01_RXNAVIMAGE10_36231B28.jpg,00093-3165-01_NLMIMAGE10_19270CA8.jpg,image/images/gallery/120/00093-3165-01_RXNAVIM...,image/images/gallery/120/00093-3165-01_NLMIMAG...,image/images/gallery/300/00093-3165-01_RXNAVIM...,image/images/gallery/300/00093-3165-01_NLMIMAG...,image/images/gallery/600/00093-3165-01_RXNAVIM...,image/images/gallery/600/00093-3165-01_NLMIMAG...,image/images/gallery/800/00093-3165-01_RXNAVIM...,image/images/gallery/800/00093-3165-01_NLMIMAG...,image/images/gallery/1024/00093-3165-01_RXNAVI...,image/images/gallery/1024/00093-3165-01_NLMIMA...,image/images/gallery/original/00093-3165-01_RX...,image/images/gallery/original/00093-3165-01_NL...
2,00093-0810-01,198045,Nortriptyline 10 MG Oral Capsule,185646437,00093-0810-01_RXNAVIMAGE10_24231228.jpg,00093-0810-01_NLMIMAGE10_34271A58.jpg,image/images/gallery/120/00093-0810-01_RXNAVIM...,image/images/gallery/120/00093-0810-01_NLMIMAG...,image/images/gallery/300/00093-0810-01_RXNAVIM...,image/images/gallery/300/00093-0810-01_NLMIMAG...,image/images/gallery/600/00093-0810-01_RXNAVIM...,image/images/gallery/600/00093-0810-01_NLMIMAG...,image/images/gallery/800/00093-0810-01_RXNAVIM...,image/images/gallery/800/00093-0810-01_NLMIMAG...,image/images/gallery/1024/00093-0810-01_RXNAVI...,image/images/gallery/1024/00093-0810-01_NLMIMA...,image/images/gallery/original/00093-0810-01_RX...,image/images/gallery/original/00093-0810-01_NL...
3,00093-0811-01,317136,Nortriptyline 25 MG Oral Capsule,185646420,00093-0811-01_RXNAVIMAGE10_20231018.jpg,00093-0811-01_NLMIMAGE10_15270A98.jpg,image/images/gallery/120/00093-0811-01_RXNAVIM...,image/images/gallery/120/00093-0811-01_NLMIMAG...,image/images/gallery/300/00093-0811-01_RXNAVIM...,image/images/gallery/300/00093-0811-01_NLMIMAG...,image/images/gallery/600/00093-0811-01_RXNAVIM...,image/images/gallery/600/00093-0811-01_NLMIMAG...,image/images/gallery/800/00093-0811-01_RXNAVIM...,image/images/gallery/800/00093-0811-01_NLMIMAG...,image/images/gallery/1024/00093-0811-01_RXNAVI...,image/images/gallery/1024/00093-0811-01_NLMIMA...,image/images/gallery/original/00093-0811-01_RX...,image/images/gallery/original/00093-0811-01_NL...
4,00093-0812-01,198046,Nortriptyline 50 MG Oral Capsule,185646464,00093-0812-01_RXNAVIMAGE10_2D2316D8.jpg,00093-0812-01_NLMIMAGE10_DD0E6EE3.jpg,image/images/gallery/120/00093-0812-01_RXNAVIM...,image/images/gallery/120/00093-0812-01_NLMIMAG...,image/images/gallery/300/00093-0812-01_RXNAVIM...,image/images/gallery/300/00093-0812-01_NLMIMAG...,image/images/gallery/600/00093-0812-01_RXNAVIM...,image/images/gallery/600/00093-0812-01_NLMIMAG...,image/images/gallery/800/00093-0812-01_RXNAVIM...,image/images/gallery/800/00093-0812-01_NLMIMAG...,image/images/gallery/1024/00093-0812-01_RXNAVI...,image/images/gallery/1024/00093-0812-01_NLMIMA...,image/images/gallery/original/00093-0812-01_RX...,image/images/gallery/original/00093-0812-01_NL...


In [ ]:
import pandas as pd
import os
import time # Import time for sleep

ROOT_DIR = "/content/drive/MyDrive/Datasets/data/"
CSV_PATH = os.path.join(ROOT_DIR, "table.csv")

# Diagnostic check
print(f"Re-checking if ROOT_DIR exists: {os.path.exists(ROOT_DIR)}")
if os.path.exists(ROOT_DIR):
    print(f"Contents of {ROOT_DIR}: {os.listdir(ROOT_DIR)}")
else:
    print(f"ROOT_DIR {ROOT_DIR} still does not exist!")

print(f"Re-checking if CSV_PATH exists: {os.path.exists(CSV_PATH)}")

# Add a small delay in case of transient mount issues
try:
    time.sleep(5)
    df = pd.read_csv(CSV_PATH)
except FileNotFoundError as e:
    print(f"Critical Error: CSV file not found at {CSV_PATH} after delay. Details: {e}")
    # Attempt to re-list contents for final diagnosis
    if os.path.exists(ROOT_DIR):
        print(f"Contents of {ROOT_DIR} during error: {os.listdir(ROOT_DIR)}")
    else:
        print(f"ROOT_DIR {ROOT_DIR} does not exist during error.")
    raise # Re-raise the exception after printing diagnostics
except Exception as e:
    print(f"An unexpected error occurred while reading the CSV: {e}")
    raise

IMG_COL = "RXNAV ORIGINAL" # Changed to use original images
LABEL_COL = "name" # drugname


assert IMG_COL in df.columns, f"{IMG_COL} not found in csv columns {df.columns}"
assert LABEL_COL in df.columns, f"{LABEL_COL} not found in csv columns {df.columns}"

df = df.dropna(subset=[IMG_COL, LABEL_COL]).reset_index(drop = True)

labels = sorted(df[LABEL_COL].unique())
label_to_id = {lbl: idx for idx, lbl in enumerate(labels)}
id_2_lbl = {idx: lbl for lbl, idx in label_to_id.items()}

df["label_id"] = df[LABEL_COL].map(label_to_id)

print(f"Unique labels:{len(labels)}")
display(df[[IMG_COL, LABEL_COL, "label_id"]].head())

Re-checking if ROOT_DIR exists: True
Contents of /content/drive/MyDrive/Datasets/data/: ['table.csv', 'readme', 'image', 'MONGOexport']
Re-checking if CSV_PATH exists: True
Unique labels:2111


,RXNAV ORIGINAL,name,label_id
0,image/images/gallery/original/00093-0311-01_RX...,Loperamide Hydrochloride 2 MG Oral Capsule,1052
1,image/images/gallery/original/00093-3165-01_RX...,Minocycline 50 MG Oral Capsule,1137
2,image/images/gallery/original/00093-0810-01_RX...,Nortriptyline 10 MG Oral Capsule,1193
3,image/images/gallery/original/00093-0811-01_RX...,Nortriptyline 25 MG Oral Capsule,1194
4,image/images/gallery/original/00093-0812-01_RX...,Nortriptyline 50 MG Oral Capsule,1195


In [ ]:
from torch.utils.data import Dataset # Added missing import
import os # Make sure os is imported for path operations

class C3PIPillDataset(Dataset):
    def __init__(self, dataframe, root_img_dir, transform=None,
                 img_col="RXNAV 120", label_col="label_id"):
        self.df = dataframe.reset_index(drop=True)
        self.root_dir = root_img_dir
        self.transform = transform
        self.img_col = img_col
        self.label_col = label_col

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        # Get the path from CSV: e.g. "image/images/gallery/120/xxx.jpg"
        rel_path = row[self.img_col]

        potential_img_paths = []

        # Attempt 1: Strip "image/" from rel_path, assuming root_dir is the base 'data' directory.
        # This makes sense if rel_path starts with "image/" and root_dir does not contain it.
        cleaned_rel = rel_path
        if rel_path.startswith("image/"):
            cleaned_rel = rel_path[6:] # strip "image/"
        potential_img_paths.append(os.path.join(self.root_dir, cleaned_rel))

        # Attempt 2: Strip "image/images/" from rel_path
        # This handles cases where "images/" is directly under the root_dir
        alt_rel = rel_path.replace("image/images/", "")
        potential_img_paths.append(os.path.join(self.root_dir, alt_rel))

        # Attempt 3: Use rel_path as is (if root_dir is exactly where rel_path starts)
        potential_img_paths.append(os.path.join(self.root_dir, rel_path))

        final_img_path = None
        for p in potential_img_paths:
            if os.path.exists(p):
                final_img_path = p
                break

        if final_img_path is None:
            print(f"Could not find image for rel_path: {rel_path}")
            print(f"Tried paths: {potential_img_paths}")

            # Add diagnostic prints for the 'image' subdirectories
            image_subdir_path = os.path.join(self.root_dir, 'image')
            if os.path.exists(image_subdir_path) and os.path.isdir(image_subdir_path):
                print(f"Contents of '{image_subdir_path}': {os.listdir(image_subdir_path)}")

                images_subdir_path = os.path.join(image_subdir_path, 'images')
                if os.path.exists(images_subdir_path) and os.path.isdir(images_subdir_path):
                    print(f"Contents of '{images_subdir_path}': {os.listdir(images_subdir_path)}")
                else:
                    print(f"The directory '{images_subdir_path}' does not exist or is not a directory.")
            else:
                print(f"The directory '{image_subdir_path}' does not exist or is not a directory.")

            raise FileNotFoundError(f"Image not found for {rel_path}")

        try:
            img = Image.open(final_img_path).convert("RGB")
        except Exception as e:
            print(f"Error opening image {final_img_path}: {e}")
            raise

        if self.transform:
            img = self.transform(img)

        label = int(row[self.label_col])
        return img, label

In [ ]:
df_filtered = df.groupby("label_id").filter(lambda x: len(x) > 1).reset_index(drop=True)
print("Before filtering:", len(df))
print("After filtering:", len(df_filtered))
df = df_filtered

Before filtering: 4332
After filtering: 3136


In [ ]:
train_df, val_df = train_test_split(
    df,
    test_size=0.2,

    random_state=42
)

print("Training samples:", len(train_df))
print("Validation samples:", len(val_df))

Training samples: 2508
Validation samples: 628


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import os
import pandas as pd
import json
import requests
from PIL import Image
from io import BytesIO
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Standard ImageNet normalization values
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

# Stronger Data Augmentation to fight overfitting
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)), # Added RandomResizedCrop
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(30), # Increased rotation from 20 to 30
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1), # Increased color jitter
    transforms.RandomAffine(degrees=0, translate=(0.15, 0.15), scale=(0.85, 1.15), shear=10), # Increased translate and scale, added shear
    transforms.RandomPerspective(distortion_scale=0.3, p=0.5), # Added RandomPerspective
    transforms.ToTensor(),
    normalize
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    normalize
])

In [ ]:
# Ensure IMG_DIR is defined. It usually comes from earlier cells.
if 'IMG_DIR' not in locals():
    # Fallback if IMG_DIR was lost in state
    IMG_DIR = "/content/drive/MyDrive/Datasets/data/"

# Pass img_col=IMG_COL to ensure we use the column selected by the user (RXNAV ORIGINAL)
train_dataset = C3PIPillDataset(train_df, IMG_DIR, transform= train_transform, img_col=IMG_COL)
val_dataset   = C3PIPillDataset(val_df,   IMG_DIR, transform= val_transform, img_col=IMG_COL)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False)

print(f"Train dataset size: {len(train_dataset)}")
print(f"Val dataset size: {len(val_dataset)}")

Train dataset size: 2508
Val dataset size: 628


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
print("Google Drive re-mounted.")

Mounted at /content/drive
Google Drive re-mounted.


In [ ]:
import os

ROOT_DIR = "/content/drive/MyDrive/Datasets/data/"
CSV_PATH = os.path.join(ROOT_DIR, "table.csv")

print(f"Checking if ROOT_DIR exists: {os.path.exists(ROOT_DIR)}")
if os.path.exists(ROOT_DIR):
    print(f"Contents of ROOT_DIR ({ROOT_DIR}): {os.listdir(ROOT_DIR)}")
else:
    print(f"ROOT_DIR does not exist. Listing contents of /content/drive/MyDrive/:")
    try:
        print(os.listdir('/content/drive/MyDrive/'))
    except Exception as e:
        print(f"Could not list /content/drive/MyDrive/: {e}")

print(f"Checking if CSV_PATH exists: {os.path.exists(CSV_PATH)}")

Checking if ROOT_DIR exists: True
Contents of ROOT_DIR (/content/drive/MyDrive/Datasets/data/): ['table.csv', 'readme', 'image', 'MONGOexport']
Checking if CSV_PATH exists: True


In [ ]:
num_classes = len(labels)

model = models.mobilenet_v2(weights = "IMAGENET1K_V1")

# Increase Dropout to 0.5 to further fight overfitting
model.classifier[0] = nn.Dropout(p=0.5)

model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

print(model.classifier)

Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth


100%|██████████| 13.6M/13.6M [00:00<00:00, 167MB/s]

Sequential(
  (0): Dropout(p=0.5, inplace=False)
  (1): Linear(in_features=1280, out_features=2111, bias=True)
)


In [ ]:
def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0

    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        # Accumulate loss (multiply by batch size to get total loss for the batch)
        running_loss += loss.item() * images.size(0)

    # Return average loss over the dataset
    return running_loss / len(loader.dataset)

def evaluate(model, loader, device):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Return accuracy
    return correct / total

In [ ]:
EPOCHS = 45

history = {"train_loss": [], "val_acc": []}

for epoch in range(EPOCHS):
    train_loss = train_one_epoch(model, train_loader, optimizer, criterion, device)
    val_acc    = evaluate(model, val_loader, device)

    history["train_loss"].append(train_loss)
    history["val_acc"].append(val_acc)

    print(f"Epoch {epoch+1}/{EPOCHS} | "
          f"Train Loss: {train_loss:.4f} | "
          f"Val Acc: {val_acc*100:.2f}%")

Epoch 1/45 | Train Loss: 7.6175 | Val Acc: 0.32%
Epoch 2/45 | Train Loss: 6.9726 | Val Acc: 0.00%
Epoch 3/45 | Train Loss: 6.5767 | Val Acc: 0.00%
Epoch 4/45 | Train Loss: 6.3113 | Val Acc: 0.32%
Epoch 5/45 | Train Loss: 6.1063 | Val Acc: 0.16%
Epoch 6/45 | Train Loss: 5.9365 | Val Acc: 0.64%
Epoch 7/45 | Train Loss: 5.8363 | Val Acc: 1.59%
Epoch 8/45 | Train Loss: 5.6845 | Val Acc: 1.11%
Epoch 9/45 | Train Loss: 5.6406 | Val Acc: 1.27%
Epoch 10/45 | Train Loss: 5.5312 | Val Acc: 1.43%
Epoch 11/45 | Train Loss: 5.5527 | Val Acc: 0.96%
Epoch 12/45 | Train Loss: 5.4062 | Val Acc: 1.91%
Epoch 13/45 | Train Loss: 5.2554 | Val Acc: 1.75%
Epoch 14/45 | Train Loss: 5.1719 | Val Acc: 1.91%
Epoch 15/45 | Train Loss: 5.0877 | Val Acc: 2.71%
Epoch 16/45 | Train Loss: 4.9867 | Val Acc: 3.34%
Epoch 17/45 | Train Loss: 4.8778 | Val Acc: 3.34%
Epoch 18/45 | Train Loss: 4.7987 | Val Acc: 3.66%
Epoch 19/45 | Train Loss: 4.7432 | Val Acc: 3.03%
Epoch 20/45 | Train Loss: 4.6485 | Val Acc: 3.50%
Epoch 21/

In [ ]:
import matplotlib.pyplot as plt

# Calculate images per class
class_counts = df['label_id'].value_counts()

print(f"Total Classes: {len(class_counts)}")
print(f"Average images per class: {class_counts.mean():.2f}")
print(f"Median images per class: {class_counts.median():.2f}")
print(f"Max images in a single class: {class_counts.max()}")
print(f"Classes with only 2 images: {sum(class_counts == 2)}")

# Visualize
plt.figure(figsize=(10, 5))
class_counts.hist(bins=range(1, 20), edgecolor='black')
plt.title("Distribution of Images per Class")
plt.xlabel("Number of Images")
plt.ylabel("Number of Classes")
plt.xticks(range(1, 20))
plt.grid(axis='y', alpha=0.5)
plt.show()

In [ ]:
plt.figure(figsize=(10,4))

plt.subplot(1,2,1)
plt.plot(history["train_loss"])
plt.title("Training Loss")

plt.subplot(1,2,2)
plt.plot([x*100 for x in history["val_acc"]])
plt.title("Validation Accuracy (%)")

plt.show()